In [1]:
import sys
import numpy as np
import os
current_directory = os.getcwd()
relative_path1 = '../build/env_simulator'
relative_path2 = '../build/cilqr'
file_path1 = os.path.join(current_directory, relative_path1)
file_path2 = os.path.join(current_directory, relative_path2)
sys.path.append(file_path1)
sys.path.append(file_path2)
import pybind_env_simulator

relative_path3 = '../env_simulator'
file_path3 = os.path.join(current_directory, relative_path3)
sys.path.append(file_path3)
from py_lib import py_env_sim
from py_lib import cilqr_plot
import pybind_ilqr
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import WheelZoomTool
import json

In [2]:
from IPython.display import display, HTML
display(HTML("<style> div.output_scroll { height: auto !important; max-height: none !important; } </style>"))

### Vehicle plus model test

In [3]:

param=pybind_ilqr.ILQRParam()
param.delta_t = 0.2
param.horizon =  30
param.max_iter_num = 100
param.tol = 1e-3
param.kappa_thr = 1e-5

vehicle_model = pybind_ilqr.VehicleModelBicycle()
vehicle_model.update_parameter(param)

param2=pybind_ilqr.ILQRParam()
param2.delta_t = 0.2
param2.horizon =  30
param2.max_iter_num = 100
param2.tol = 1e-3
param2.kappa_thr = 1e-5
param2.state_size = 6
param2.action_size = 2
vehicle_model_plus = pybind_ilqr.VehicleModelBicyclePlus()
vehicle_model_plus.update_parameter(param2)
#state: x, y, v, theta
state_list = []
state_list_plus = []
#action:a,delta
action_list = []
action_list_w = []
action_list_plus = []

current_path = os.getcwd()
path = current_path + "/data"
for filename in os.listdir(path):
    if filename.endswith('.json'):  # 确保文件是以 .json 结尾的 JSON 文件
        file_path = os.path.join(path, filename)

        # 读取 JSON 文件
        with open(file_path, 'r') as file:
            data = json.load(file)

        # state : x, y, v, theta
        for data_frame in data:
            for key in data_frame:
                temp_state = []
                
                temp_state.append(data_frame[key]['pos_x'])
                temp_state.append(data_frame[key]['pos_y'])
                temp_state.append(data_frame[key]['v'])
                temp_state.append(data_frame[key]['theta'])
                state_list.append(temp_state)

                temp_action = []
                temp_action.append(data_frame[key]['a'])
                temp_action.append(data_frame[key]['w'])
                action_list_w.append(temp_action)
                
                temp_state_plus = []
                temp_state_plus.append(data_frame[key]['pos_x'])
                temp_state_plus.append(data_frame[key]['pos_y'])
                temp_state_plus.append(data_frame[key]['v'])
                temp_state_plus.append(data_frame[key]['theta'])
                temp_state_plus.append(data_frame[key]['a'])
                temp_state_plus.append(data_frame[key]['w'])
                state_list_plus.append(temp_state_plus)

for i in range(len(state_list_plus)-1):
    temp_action = []
    temp_action.append((state_list_plus[i+1][4] - state_list_plus[i][4]) / 0.2)
    temp_action.append((state_list_plus[i+1][5] - state_list_plus[i][5]) / 0.2)
    action_list_plus.append(temp_action)
    
state_input=state_list[0]
next_state_tmp = []
next_state_list =[]
for i in range(len(state_list)):
    next_state_tmp=vehicle_model.step_std(state_input, action_list_w[i])
    next_state_list.append(next_state_tmp)
    state_input=next_state_tmp

state_input=state_list_plus[0]
next_state_tmp = []
next_state_list_plus =[]
for i in range(len(state_list) -1):
    next_state_tmp=vehicle_model_plus.step_std(state_input, action_list_plus[i])
    next_state_list_plus.append(next_state_tmp)
    state_input=next_state_tmp
    
# # output plot data
next_x_list = []
next_y_list = []
for next_state in next_state_list:
    next_x_list.append(next_state[0])
    next_y_list.append(next_state[1])

    
next_x_list_plus = []
next_y_list_plus = []
for next_state in next_state_list_plus:
    next_x_list_plus.append(next_state[0])
    next_y_list_plus.append(next_state[1])
# # input plot data
state_x = []
state_y = []
state_v = []
state_theta = []

for state in state_list:
    state_x.append(state[0])
    state_y.append(state[1])
    state_v.append(state[2])
    state_theta.append(state[3])

fig_x_y = figure(width=800, height=400, match_aspect=True)

fig_x_y.scatter(next_x_list, next_y_list, color='red',
                line_alpha=0.4, legend_label="model_normal")

fig_x_y.scatter(next_x_list_plus, next_y_list_plus, color='black',
                line_alpha=0.4, legend_label="model_plis")

fig_x_y.scatter(state_x, state_y, color='green',
                line_alpha=0.4, legend_label="x_real-y_real")

fig_x_y.xaxis.axis_label = "x"
fig_x_y.yaxis.axis_label = "y"

fig_x_y.toolbar.active_scroll = fig_x_y.select_one(WheelZoomTool)
fig_x_y.legend.click_policy = 'hide'
output_notebook()
show(fig_x_y)
# # print("output: ", next_state_list)

Loading BokehJS ...

In [4]:
from sympy import symbols, diff, sin, cos, exp
x = symbols('x')
y = symbols('y')
v = symbols('v')
theta = symbols('theta')
a = symbols('a')
omega = symbols('omega')
j = symbols('j')
k = symbols('k')

t = symbols('t')

x_n = x + 0.5 *( cos(theta) + cos(theta + t * omega))* (v * t + 0.5* a* t *t)
y_n = y + 0.5 *( sin(theta) + sin(theta + t * omega))* (v * t + 0.5* a* t *t)
f_x_1 = diff(x_n, x)
print("f_x_1",f_x_1)
f_x_2 = diff(x_n, theta)
print("f_x_2",f_x_2)

f_x_1 1
f_x_2 (0.5*a*t**2 + t*v)*(-0.5*sin(theta) - 0.5*sin(omega*t + theta))


In [5]:
from sympy import log
t = symbols('t')
T = symbols('T')
S = symbols('S')
v = symbols('v')
b = - log(-(v*T -S)) / t
l_x_v = diff(b, v)
print(diff(l_x_v,v))

T**2/(t*(S - T*v)**2)


In [6]:
q1 = symbols('q1')
q2 = symbols('q2')
thr = symbols('thr')
j = symbols('j')
b_j_1 = q1 * exp(q2 * (j - thr))
b_j_2 = q1 * exp(q2 * (thr - j))
print(diff(b_j_1, j))
print(diff(b_j_2, j))

q1*q2*exp(q2*(j - thr))
-q1*q2*exp(q2*(-j + thr))


In [7]:
q1 = symbols('q1')
q2 = symbols('q2')
q3 = symbols('q3')
q4 = symbols('q4')
q5 = symbols('q5')
a = symbols('a')
g = q1* a - q2*(q2+q5*a)/q4 + 0.5* q4*(q2+q5*a)*(q2+q5*a)/(q4*q4) + 0.5 *q3*a*a
print(g)

0.5*a**2*q3 + a*q1 - q2*(a*q5 + q2)/q4 + 0.5*(a*q5 + q2)**2/q4


In [8]:
def get_use_exp_list(l_conditions):
    use_exp_list = []
    for condition in l_conditions:
        use_exp = False
#         print(condition.use_exp_model_map)
        for item in condition.use_exp_model_map:
            if condition.use_exp_model_map[item] == True:
                use_exp = True
                break
        use_exp_list.append(use_exp)
    return use_exp_list

In [9]:
def show_motion_tree_case(motion): 
    traj_trees = motion.get_trajectory_tree()

    print("trajectory tree branch size is: ", len(traj_trees))
    env = motion.get_env()
    planning_origin = motion.get_planning_origin()

    pts = env.get_all_center_points()
    obs = env.get_all_obstacle()
    planning_init = {}

    planning_init['x'] = planning_origin.position.x
    planning_init['y'] = planning_origin.position.y
    planning_init['theta'] = planning_origin.theta

    trajs = motion.get_init_trajectory()
    traj_list = []
    for traj in trajs:
        pt = {}
        pt['x'] = traj.position.x
        pt['y'] = traj.position.y
        pt['theta'] = traj.theta
        traj_list.append(pt)
    
    lon_debug = motion.get_lon_debug_tree()
    lat_debug = motion.get_lat_debug_tree()
    traj_list_list = []
    j = 0
    for traj_tree in traj_trees:  
        traj_new_list = []
        index = 0
        for traj in traj_tree:
            pt = {}
            pt['x'] = traj.position.x
            pt['y'] = traj.position.y
            pt['theta'] = traj.theta
            pt['v'] = traj.velocity
            pt['acc'] = traj.acceleration
            pt['acc_lat'] = traj.velocity * traj.omega
            pt['ref_vel'] = lon_debug[j][index].v_ref
            pt['ref_acc'] = lon_debug[j][index].a_ref
            pt['match_point'] = lat_debug[j][index].match_point
            pt['jerk'] = traj.jerk
            pt['omega_dot'] = traj.omega_dot
            index = index + 1
            traj_new_list.append(pt)
        traj_list_list.append(traj_new_list)
        j = j+1
    cilqr_plot.plot_tree_cilqr_constraint(pts,obs,planning_init,traj_list, traj_list_list,[])

# 无车 纵向场景 限速 80

In [10]:
motion = pybind_ilqr.LateralLongitudinalMotion()
point_init = pybind_ilqr.PlanningPoint()
point_init.position.x = 10.0
point_init.position.y = 4.0
point_init.theta = 0.00
point_init.velocity = 15.0
point_init.acceleration  = 0.6

path = str(current_directory)+str("/../env_simulator/data/test_case_1_straight_lanes_cruise.json")
motion.init(path,point_init)

motion.execute_tree()

show_motion_tree_case(motion)

---- loading test_case_1_straight_case_1 case ----
---- loading relative -1 lane 
---- loading relative 0 lane 
---- loading relative 1 lane 
---- loading obstacle 100
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 298.743
------- best alpha: 1
------- best alpha: 1
----------enable lat 1
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 0.909091
------- best alpha: 0.909091
 iter times: 5
 stop search 
 **************** ilqr tree time: 9.41838 ms 
 tree trajectory finish
trajectory tree branch size is:  1


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

# 有车 纵向跟车场景 1  80-> 40

In [11]:
path = str(current_directory)+str("/../env_simulator/data/test_case_2_straight_lanes_cruise.json")
point_init = pybind_ilqr.PlanningPoint()
point_init.position.x = 10.0
point_init.position.y = 4.0
point_init.theta = 0.0
point_init.velocity = 20.0
point_init.acceleration  = 0.5
motion.init(path,point_init)
motion.execute_tree()

show_motion_tree_case(motion)

trajectory tree branch size is:  1
---- loading test_case_2_straight_case_2 case ----
---- loading relative -1 lane 
---- loading relative 0 lane 
---- loading relative 1 lane 
---- loading obstacle 100
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 1.76001e+10
------- best alpha: 0.0323492
------- best alpha: 0.683013
------- best alpha: 0.909091
------- best alpha: 0.683013
------- best alpha: 0.683013
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
 iter times: 7
 stop search 
 **************** ilqr tree time: 12.6124 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

# 有车 纵向跟车场景 2  80-> 60

In [12]:
path = str(current_directory)+str("/../env_simulator/data/test_case_3_straight_lanes_cruise.json")
point_init = pybind_ilqr.PlanningPoint()
point_init.position.x = 10.0
point_init.position.y = 4.0
point_init.theta = 0.0
point_init.velocity = 23.0
point_init.acceleration  = 0.5
motion.init(path,point_init)
motion.execute_tree()

show_motion_tree_case(motion)

---- loading test_case_2_straight_case_2 case ----
trajectory tree branch size is:  1
---- loading relative -1 lane 
---- loading relative 0 lane 
---- loading relative 1 lane 
---- loading obstacle 100
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 1.22001e+10
------- best alpha: 0.0323492
------- best alpha: 0.683013
------- best alpha: 0.683013
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 1
------- best alpha: 1
------- best alpha: 1
 iter times: 7
 stop search 
 **************** ilqr tree time: 12.7404 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

# 有车 纵向跟车场景 3  80-> 80

In [13]:
path = str(current_directory)+str("/../env_simulator/data/test_case_4_straight_lanes_cruise.json")
point_init = pybind_ilqr.PlanningPoint()
point_init.position.x = 10.0
point_init.position.y = 4.0
point_init.theta = 0.0
point_init.velocity = 22.22
point_init.acceleration  = 0.6
motion.init(path,point_init)
motion.execute_tree()

show_motion_tree_case(motion)

trajectory tree branch size is:  1
---- loading test_case_2_straight_case_2 case ----
---- loading relative -1 lane 
---- loading relative 0 lane 
---- loading relative 1 lane 
---- loading obstacle 100
Field 'road_edges' does not exist.
 r_4 is : 1.13033
j_opt: 3.30002e+09
------- best alpha: 0.683013
------- best alpha: 0.683013
------- best alpha: 0.909091
------- best alpha: 1
------- best alpha: 0.683013
------- best alpha: 0.909091
------- best alpha: 0.909091
 iter times: 6
 stop search 
 **************** ilqr tree time: 10.5899 ms 
 tree trajectory finish


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...